In [ ]:
# Inital state probability matrix, given
PI = [0.5,0.5]

# state transition probability matrix, given
A = [[0.8,0.2],[0.001,0.999]]

# Emission or observation probability matrix, given
B = [[0.5,0.1,0.3,0.1,0.0],[0.05,0.35,0.2,0.2,0.2]]

# state codes of the observed states, dummy
states = [0, 1]

#Observables Movement:0 Passive Social:1 Active Social:2 Texting:3 Access Psych Site:4
Observables = [0,3,0,3]

# Filtering

In [3]:
import numpy as np 
from numpy import matrix

def forward(PI, A, B, Observables):
    alpha_matrix = []
    curr = {}
    
    for i in range (0, len(A)):
        curr[i] = PI[i] * B[i][0]
    
    alpha_matrix.append(list(curr.values()))

    for t in range (1, len(Observables)):
        
        nex = {}
        for i in range (len(A)):
            t_value = Observables[t]
            nex[i] =  sum(curr[j] * A[j][i] * B[i][t_value] for j in range(len(A)))

        alpha_matrix.append(list(nex.values()))

        curr = nex.copy()
            
    alpha_matrix = np.transpose(alpha_matrix)

    return alpha_matrix

In [4]:
def forward_normalized(PI, A, B, Observables):
    alpha_matrix = []
    curr = {}
    c = {}
    for i in range (0, len(A)):
        curr[i] = PI[i] * B[i][0]
        
    c[0] = sum(curr[i] for i in range(len(curr)))

    
    c[0] = 1/c[0]
    for i in range (0, len(A)):
        curr[i] = c[0] * curr[i]
    
    alpha_matrix.append(list(curr.values()))

    
    for t in range (1, len(Observables)):
        nex = {}
        nex_norm = {}
        for i in range (len(A)):
            t_value = Observables[t]
            nex[i] =  sum(curr[j] * A[j][i] * B[i][t_value] for j in range(len(A)))
        c[t] = 1/sum(nex[k] for k in range(len(nex)))
        for i in range (0,len(A)):
            nex_norm[i] = c[t] * nex[i]
        alpha_matrix.append(list(nex_norm.values()))
        curr = nex_norm
        
      
            
    alpha_matrix = np.transpose(alpha_matrix)
    c_array = list(c.values())

    return alpha_matrix, c_array

# Evaluation

In [5]:
def evaluation_unnormalized(alpha):
    probability = 0
    for i in range(len(alpha)):
        for j in range(0, len(alpha[i])):
             probability += alpha[i][j]        
    return probability

In [6]:
import math
def evaluation_normalized(c):
    probability = -1 * sum(math.log(i) for i in c)
    return probability

# Backward Probability

In [7]:
def backward(PI, A, B, Observables):
    
    beta = []
    curr = {}

    b_prev = {}
    for i, value in enumerate(reversed(Observables)):
        b_curr = {}
        for state in states:
            if i == 0:
                b_curr[state] = 1 
            else:
                b_curr[state] = sum(A[state][l] * B[l][value] * b_prev[l] for l in states)

        beta.insert(0,list(b_curr.values()))
        b_prev = b_curr

    
    return np.transpose(beta) 

In [8]:
def backward_normalized(PI, A, B, c, Observables):
    beta = []
    curr = {}

    b_prev = {}
    for i, value in enumerate(reversed(Observables)):
        b_curr = {}
        for state in states:
            if i == 0:
                b_curr[state] = 1 
                b_curr[state] = b_curr[state] * c[i]
            else:
                b_curr[state] = sum(A[state][l] * B[l][value] * b_prev[l] for l in states)

        beta.insert(0,list(b_curr.values()))
        b_prev = b_curr
            
    return np.transpose(beta)

In [9]:
alpha = forward(PI,A,B,Observables)

In [10]:
alpha_norm, c = forward_normalized(PI, A, B, Observables)

In [11]:
print("c matrix")
print(c)

c matrix
[3.6363636363636362, 7.857704121722981, 3.9070511989489494, 7.784209968703886]


In [12]:
print("Alpha matrix")
print(alpha)

Alpha matrix
[[0.25       0.0200025  0.0080085  0.00064077]
 [0.025      0.014995   0.00094903 0.00050996]]


In [13]:
beta = backward(PI, A, B, Observables)

In [14]:
print("Beta Matrix")
print(beta)

Beta Matrix
[[1.40284091e-02 3.48180000e-02 4.10000000e-01 1.00000000e+00]
 [5.22948455e-04 1.01209100e-02 5.04500000e-02 1.00000000e+00]]


In [21]:
print("Backward normalized matrix")
print(backward_normalized(PI, A, B, c, Observables))
print("Alpha normalized matrix")
print(alpha_norm)

Backward normalized matrix
[[5.10123967e-02 1.26610909e-01 1.49090909e+00 3.63636364e+00]
 [1.90163074e-03 3.68033091e-02 1.83454545e-01 3.63636364e+00]]
Alpha normalized matrix
[[0.90909091 0.57154082 0.89405271 0.55684199]
 [0.09090909 0.42845918 0.10594729 0.44315801]]


In [22]:
print("Evaluation unnormalized")
print(evaluation_unnormalized(alpha))
print("Evaluation normalized")
print(evaluation_normalized(c))

Evaluation unnormalized
0.320105752597475
Evaluation normalized
-6.767358887933846


# Smoothing by Forward-Backward Algorithm

In [16]:
def smoothed_probability(alpha, beta, noOfStates, noOfTimeSteps):
    matrix = []
    for i in range(len(alpha)):
        curr = {}
        for t in range(len(alpha[i])):
            curr[t] = alpha[i][t] * beta[i][t] / (sum(alpha[j][t] * beta[j][t] for j in range(len(alpha))))
        matrix.append(list(curr.values()) ) 
                
    return matrix

In [17]:
print("Smoothed probability matrix")
smoothed_probability(alpha,beta,len(alpha), len(beta))

Smoothed probability matrix


[[0.9962860631362895,
  0.8210784719980339,
  0.9856280045317052,
  0.5568419937407776],
 [0.003713936863710455,
  0.17892152800196623,
  0.014371995468294799,
  0.44315800625922225]]